In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [ ]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [3]:
DROP TABLE IF EXISTS interacao;

CREATE VIEW interacao AS
SELECT DU1.codeDrug drug1, DU2.codeDrug drug2, COUNT (*) num
FROM DrugUse DU1, DrugUse DU2
WHERE DU1.idPerson = DU2.idPerson AND DU1.codeDrug < DU2.codeDrug
GROUP BY DU1.codeDrug, DU2.codeDrug;

SELECT * FROM interacao WHERE num > 30;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [ ]:
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;

In [ ]:
SELECT * FROM DrugInteraction WHERE weight > 30;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [5]:
SELECT DI.druga, DI.drugb, cast(DI.weight AS DOUBLE)/(SELECT SUM(weight) FROM DrugInteraction) suporte
FROM (SELECT * FROM DrugInteraction WHERE weight > 30) DI
GROUP BY DI.druga, DI.drugb
ORDER BY suporte;

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [6]:
DROP TABLE IF EXISTS total_interactions;

CREATE VIEW total_interactions AS
SELECT total_a.druga, (total_a.soma + total_b.soma) soma
FROM (SELECT druga, SUM(weight) soma
      FROM DrugInteraction 
      GROUP BY druga) total_a,
     (SELECT drugb, SUM(weight) soma
      FROM DrugInteraction 
      GROUP BY drugb) total_b
WHERE total_a.druga = total_b.drugb
GROUP BY total_a.druga;

In [7]:
SELECT DI.druga, DI.drugb, cast(DI.weight AS DOUBLE)/TI.soma confianca
FROM (SELECT * FROM DrugInteraction WHERE weight > 30) DI, total_interactions TI
WHERE DI.druga = TI.druga 
GROUP BY DI.druga, DI.drugb
ORDER BY confianca;

In [8]:
SELECT DI.drugb, DI.druga, cast(DI.weight AS DOUBLE)/TI.soma confianca
FROM (SELECT * FROM DrugInteraction WHERE weight > 30) DI, total_interactions TI
WHERE DI.drugb = TI.druga 
GROUP BY DI.druga, DI.drugb
ORDER BY confianca;

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [ ]:
CREATE VIEW colateral AS
SELECT DU.idPerson Pessoa, DU.codeDrug
FROM DrugUse DU, SideEffect SE
WHERE DU.idPerson = SE.idPerson

In [ ]:
DROP TABLE IF EXISTS drug_a_users;
DROP TABLE IF EXISTS drug_b_users;

CREATE VIEW drug_a_users AS
SELECT DISTINCT c1.Pessoa, c1.codeDrug
FROM colateral c1, (SELECT * FROM DrugInteraction WHERE weight > 30) interacao
WHERE c1.codeDrug = interacao.druga;

CREATE VIEW drug_b_users AS
SELECT DISTINCT c1.Pessoa, c1.codeDrug
FROM colateral c1, (SELECT * FROM DrugInteraction WHERE weight > 30) interacao
WHERE c1.codeDrug = interacao.drugb;

In [ ]:
DROP TABLE IF EXISTS uso_conjunto;

CREATE VIEW uso_conjunto AS
SELECT da.Pessoa, da.codeDrug d1, db.codeDrug d2
FROM  drug_a_users da, drug_b_users db, (SELECT * FROM DrugInteraction WHERE weight > 30) DI
WHERE da.codeDrug = DI.druga AND db.codeDrug = DI.drugb
GROUP BY da.Pessoa, db.codeDrug, da.codeDrug;

SELECT * FROM uso_conjunto;